<a href="https://colab.research.google.com/github/akuma527/IProjects/blob/master/I%26D_Whale/Model_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
pip install -U tqdm

     |████████████████████████████████| 61kB 2.1MB/s 
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [3]:
%cd /content/drive/My Drive/Colab Notebooks/I&D_Whale/data/

/content/drive/My Drive/Colab Notebooks/I&D_Whale/data


## Loading a Pretrained VGG16

In [0]:
import torch
import torchvision.models as models
from torchvision import transforms
from torch.autograd import Variable
from PIL import Image

In [5]:
model = models.vgg16(pretrained=True, progress=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


Image Preprocessing Function

In [0]:
imsize = 256
loader = transforms.Compose([transforms.Resize(imsize), transforms.ToTensor()])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name)
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image #assumes that you're using GPU

In [0]:
import pandas as pd
import os
import cv2

In [0]:
t1 = pd.read_csv('final_train.csv')
t2 = pd.read_csv('test.csv')

In [9]:
t2.head()

,Image,Id,fil_path
0,PM-WWA-20170321-046.jpg,0,test/PM-WWA-20170321-046.jpg
1,PM-WWA-20170321-053.jpg,0,test/PM-WWA-20170321-053.jpg
2,PM-WWA-20170321-322.jpg,0,test/PM-WWA-20170321-322.jpg
3,PM-WWA-20170321-332.jpg,0,test/PM-WWA-20170321-332.jpg
4,PM-WWA-20170321-344.jpg,0,test/PM-WWA-20170321-344.jpg


In [10]:
t1.head()

,Image,Id,fil_path,Image_2,Id_2,fil_path_2
0,PM-WWA-20050413-003b.jpg,2764,train/2764/PM-WWA-20050413-003b.jpg,PM-WWA-20050413-029.jpg,2764,train/2764/PM-WWA-20050413-029.jpg
1,PM-WWA-20050413-029.jpg,2764,train/2764/PM-WWA-20050413-029.jpg,PM-WWA-20050413-003b.jpg,2764,train/2764/PM-WWA-20050413-003b.jpg
2,PM-WWA-20050413-006.jpg,2825,train/2825/PM-WWA-20050413-006.jpg,PM-WWA-20050413-007.jpg,2825,train/2825/PM-WWA-20050413-007.jpg
3,PM-WWA-20050413-007.jpg,2825,train/2825/PM-WWA-20050413-007.jpg,PM-WWA-20050413-006.jpg,2825,train/2825/PM-WWA-20050413-006.jpg
4,PM-WWA-20050413-009.jpg,2826,train/2826/PM-WWA-20050413-009.jpg,PM-BIO-20050413-049.jpg,2826,train/2826/PM-BIO-20050413-049.jpg


In [11]:
PATH_TO_IMAGE = t1['fil_path'].iloc[2]
PATH_TO_IMAGE

'train/2825/PM-WWA-20050413-006.jpg'

In [0]:
image = image_loader(PATH_TO_IMAGE)


In [0]:
# if torch.cuda.is_available():
#     model.cuda()

In [0]:
# model(image).data.numpy().argmax() 

In [0]:
def get_labels():
    LABELS_URL = 'https://s3.amazonaws.com/outcome-blog/imagenet/labels.json'

    import requests
    # Let's get our class labels.
    response = requests.get(LABELS_URL)  # Make an HTTP GET request and store the response.
    labels = {int(key): value for key, value in response.json().items()}
    return labels

In [0]:
# labels[5]

Removing Classification Layer

In [0]:
model.classifier = torch.nn.Sequential(*[model.classifier[i] for i in range(4)])

In [0]:
PATH_TO_IMAGE_1 = t1['fil_path'].iloc[1]
image_1 = image_loader(PATH_TO_IMAGE_1)
PATH_TO_IMAGE_2 = t1['fil_path'].iloc[2]
image_2 = image_loader(PATH_TO_IMAGE_2)

In [0]:
v1 = model(image_1)
v2 = model(image_2)

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
cosine_similarity(v1.detach().numpy().flatten().reshape(1,-1), v2.detach().numpy().flatten().reshape(1,-1))

array([[0.8118341]], dtype=float32)

In [22]:
v1.detach().numpy().flatten().reshape(1,-1).shape

(1, 4096)

# Convert Images to Embeddings

In [23]:
a = t1[['Image', 'fil_path']]
a['type'] = 'train'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
b = t2[['Image', 'fil_path']]
b['type'] = 'test'

In [0]:
df = pd.concat([a, b], ignore_index=True, sort=False)

In [26]:
df.head()

,Image,fil_path,type
0,PM-WWA-20050413-003b.jpg,train/2764/PM-WWA-20050413-003b.jpg,train
1,PM-WWA-20050413-029.jpg,train/2764/PM-WWA-20050413-029.jpg,train
2,PM-WWA-20050413-006.jpg,train/2825/PM-WWA-20050413-006.jpg,train
3,PM-WWA-20050413-007.jpg,train/2825/PM-WWA-20050413-007.jpg,train
4,PM-WWA-20050413-009.jpg,train/2826/PM-WWA-20050413-009.jpg,train


In [0]:
from tqdm import tqdm_notebook

In [0]:
d = {}
for i in tqdm_notebook(range(len(df))):
    PATH_TO_IMAGE_1 = df['fil_path'].iloc[i]
    image_1 = image_loader(PATH_TO_IMAGE_1)
    v1 = model(image_1)
    d[PATH_TO_IMAGE_1] = v1